# AzureML in a Day

Learn how a data scientist uses Azure Machine Learning (Azure ML) to train a model, then use the model for prediction. This tutorial will help you become familiar with the core concepts of Azure ML and their most common usage. 

You'll learn how to submit a *command job* to run your *training script*, configured with the *job environment* necessary to run the script.

The training script handles the data preparation, then trains and registers a model. Once you have the model, you'll *deploy* it as an *endpoint*, then call the endpoint for *inferencing*.

The steps you'll take are:

> * Connect to your Azure ML workspace
> * Create your job environment
> * Create your training script
> * Create and run your command job to run the training script, configured with the appropriate job environment
> * View the output of your training script
> * Deploy the newly-trained model as an endpoint
> * Call the Azure ML endpoint for inferencing

## Prerequisites

* An Azure subscription. If you don't have an Azure subscription, [create a free account](https://aka.ms/AMLFree) before you begin.
* A working Azure ML workspace. A workspace can be created via Azure Portal, Azure CLI, or Python SDK. [Read more](https://docs.microsoft.com/azure/machine-learning/how-to-manage-workspace?tabs=python).
* An Azure Machine Learning [workspace]()
* A workspace and compute instance which you can create by  completing the [Quickstart: Get started with Azure Machine Learning](https://docs.microsoft.com/azure/machine-learning/quickstart-create-resources#create-compute-instance)

## Connect to the workspace

Before you dive in the code, you'll need to connect to your Azure ML workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We're using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` is used to handle most Azure SDK authentication scenarios. 

Reference for more available credentials if it doesn't work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/python/api/azure-identity/azure.identity?view=azure-python).

In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

If you want to use a browser to login and authenticate, you can use the following code instead. In this example, you'll use the `DefaultAzureCredential`.

In [2]:
# Handle to the workspace
from azure.ai.ml import MLClient

from azure.identity import InteractiveBrowserCredential
credential = InteractiveBrowserCredential()

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find these values:

1. In the upper right Azure Machine Learning studio toolbar, select your workspace name.
1. Copy the value for workspace, resource group and subscription ID into the code.  
1. You'll need to copy one value, close the area and paste, then come back for the next one.

![image of workspace credentials](media\find-credentials.png)

In [3]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="3d6a93ea-a2ba-40b0-bc50-32f9968e6bc6",
    resource_group_name="geris-ml-analysis-dev",
    workspace_name="mlw-gerisml-akpt-dev",
)

The result is a handler to the workspace that you'll use to manage other resources and jobs.

> [!IMPORTANT]
> Creating MLClient will not connect to the workspace. The client initialization is lazy, it will wait for the first time it needs to make a call (in the notebook below, that will happen during job environment creation).

## Create a job environment

To run your AzureML job, you'll need an [environment](https://docs.microsoft.com/azure/machine-learning/concept-environments). An environment lists the software runtime and libraries that you want installed on the compute where you’ll be training. It's similar to your Python environment on your local machine.

AzureML provides many curated or ready-made environments, which are useful for common training and inference scenarios. You can also create your own custom environments using a docker image, or a conda configuration.

In this example, you'll create a custom conda environment for your jobs, using a conda yaml file.

First, create a directory to store the file in.

In [4]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

Now, create the file in the dependencies directory. The cell below uses IPython magic to write the file into the directory you just created.

In [16]:
%%writefile {dependencies_dir}/conda.yaml
  name: model-env
  channels:
    - pytorch
    - conda-forge
  dependencies:
    - python=3.10
    - numpy=1.26.4
    - pip=24.0
    - scikit-learn=1.5.1
    - scipy=1.10
    - pandas=1.5.3
    - pytorch=2.2.1
    - pip:
      - inference-schema[numpy-support]==1.3.0
      - xlrd==2.0.1
      - mlflow
      - azureml-mlflow
      - psutil
      - tqdm
      - ipykernel~=6.29
      - matplotlib

  

Overwriting ./dependencies/conda.yaml



The specification contains some usual packages, that you'll use in your job (numpy, pip).

Reference this *yaml* file to create and register this custom environment in your workspace:

In [20]:
from azure.ai.ml.entities import Environment

custom_env_name = "ml-torch-lr"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for ML pipeline job",
    tags={"project": "geris-ml-lr", "type": "pipeline"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name ml-torch-lr is registered to workspace, the environment version is 1


## What is a command job?

You'll create an Azure ML *command job* to train a model for credit default prediction. The command job is used to run a *training script* in a specified environment on serverless compute.  You've already created the environment.  Next you'll create the training script.

The *training script* handles the data preparation, training and registering of the trained model. In this tutorial, you'll create a Python training script.

Command jobs can be run from CLI, Python SDK, or studio interface. In this tutorial, you'll use the Azure ML Python SDK v2 to create and run the command job.

After running the training job, you'll deploy the model, then use it to produce a prediction.

## Create training script

Let's start by creating the training script - the *main.py* Python file.

First create a source folder for the script:

In [21]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

This script handles the preprocessing of the data, splitting it into test and train data. It then consumes this data to train a tree based model and return the output model. 

[MLFlow](https://learn.microsoft.com/azure/machine-learning/how-to-log-mlflow-models) will be used to log the parameters and metrics during our pipeline run. 

The cell below uses IPython magic to write the training script into the directory you just created.

In [110]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tempfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import MultivariateNormal
import numpy as np

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")

    parser.add_argument("--test_end", type=str, required=False, default='2023-01-01', help="date to split train/test data")
    parser.add_argument("--epochs", required=False, default=1000, type=int)
    parser.add_argument("--n_samples", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    weekly_data = pd.read_json(args.data, orient="records")

    mlflow.log_metric("num_weeks", weekly_data.shape[0])

    import tempfile
    # Log first few rows as text artifact
    with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".txt") as tmpfile:
        tmpfile.write(weekly_data.head(5).to_string(index=False))
        tmpfile_path = tmpfile.name

    mlflow.log_artifact(tmpfile_path, artifact_path="data_preview")

    # Clean up temporary file
    os.remove(tmpfile_path)
  
    ####################
    #</prepare the data>
    ####################

    import logging
    import sys


    # # Configure logging to write to stdout (this ends up in std_log.txt in Azure ML)
    # logging.basicConfig(
    #     stream=sys.stdout,
    #     level=logging.INFO,
    #     format='%(asctime)s - %(levelname)s - %(message)s'
    # )
    # logger = logging.getLogger(__name__)

    weekly_data = weekly_data.set_index(pd.to_datetime(weekly_data['date']))
    weekly_data_train = weekly_data[weekly_data.index < pd.to_datetime(args.test_end)]
    weekly_data_test = weekly_data[weekly_data.index >= pd.to_datetime(args.test_end)]
    
    mlflow.log_metric("num_weeks_train", weekly_data_train.shape[0])
    mlflow.log_metric("num_weeks_test", weekly_data_test.shape[0])
    
    # logger.info(f"Columns in the dataset: {weekly_data.columns}")

    cols_fut_true = ['Butter_EEX_4_weeks_ahead', 'Butter_EEX_8_weeks_ahead', 'Butter_EEX_12_weeks_ahead', 'SMP_food_EEX_4_weeks_ahead', 'SMP_food_EEX_8_weeks_ahead', 'SMP_food_EEX_12_weeks_ahead']
    cols_fut_estimate = ['Fut_butter_4', 'Fut_butter_8', 'Fut_butter_12', 'Fut_smp_4', 'Fut_smp_8', 'Fut_smp_12']
    cols_x = ['Butter_EEX', 'SMP_food_EEX']

    cols_y = ['Gouda_EEX']
    cols_target = ['Gouda_EEX_4_weeks_ahead', 'Gouda_EEX_8_weeks_ahead', 'Gouda_EEX_12_weeks_ahead']

    fut_true_train = torch.tensor(weekly_data_train[cols_fut_true].values, dtype=torch.float32)
    fut_true_test = torch.tensor(weekly_data_test[cols_fut_true].values, dtype=torch.float32)

    fut_estimate_train = torch.tensor(weekly_data_train[cols_fut_estimate].values, dtype=torch.float32)
    fut_estimate_test = torch.tensor(weekly_data_test[cols_fut_estimate].values, dtype=torch.float32)

    x_train = torch.tensor(weekly_data_train[cols_x].values, dtype=torch.float32)
    x_test = torch.tensor(weekly_data_test[cols_x].values, dtype=torch.float32)
    y_train = torch.tensor(weekly_data_train[cols_y].values, dtype=torch.float32)
    y_test = torch.tensor(weekly_data_test[cols_y].values, dtype=torch.float32)

    fut_target_train = torch.tensor(weekly_data_train[cols_target].values, dtype=torch.float32)
    fut_target_test = torch.tensor(weekly_data_test[cols_target].values, dtype=torch.float32)

    dates_train  = weekly_data_train.index.values
    dates_test  = weekly_data_test.index.values

    ###################
    #</train the model>
    ###################
    class LearnableCovarianceFreeForm(nn.Module):
        def __init__(self, dim):
            super().__init__()
            # Parameterize the Cholesky decomposition of the covariance matrix
            self.lower_triangular = nn.Parameter(torch.eye(dim))

        def forward(self):
            lower = torch.tril(self.lower_triangular)
            diag = torch.diag(lower)
            lower = lower + torch.diag(torch.abs(diag) + 1e-6)
            covariance_matrix = torch.matmul(lower, lower.T)
            return covariance_matrix

        def log_prob(self, mean, actual_values):
            covariance = self.forward()
            dist = MultivariateNormal(mean, covariance)
            return dist.log_prob(actual_values)

        def sample(self, num_samples, mean):
            covariance = self.forward()
            dist = MultivariateNormal(mean, covariance)
            return dist.sample((num_samples,))

        def get_learned_covariance(self):
            return self.forward().detach()


    class SimpleLRModel(nn.Module):
        def __init__(self, input_dim, future_dim):
            super(SimpleLRModel, self).__init__()
            self.linear = nn.Linear(input_dim, 1, bias=False)
            self.covariance_learner = LearnableCovarianceFreeForm(future_dim)

        def forward(self, x):
            return self.linear(x)
        
        def combined_log_prob(self, x_future, x_future_actual, y_pred, y_true):
            # Calculate the log probability of the future estimates given the actual values
            log_prob_future = self.covariance_learner.log_prob(x_future, x_future_actual)
            # Calculate the log probability of the target variable given the future estimates
            log_prob_target = self._likelihood(y_pred, y_true)
            return log_prob_future + log_prob_target
            
        def _likelihood(self, y_pred, y_true):
            # Assuming a Gaussian likelihood for the target variable
            mse = torch.mean((y_pred - y_true) ** 2, dim=1)
            return - mse
        
        def get_learned_future_covariance(self):
            return self.covariance_learner.get_learned_covariance()

        def sample(self, num_samples, butter_fut, smp_fut, time_steps):
            pred_samples = torch.zeros(num_samples, time_steps)
            butter_samples = torch.zeros(num_samples, len(butter_fut))
            smp_samples = torch.zeros(num_samples, len(smp_fut))

            futs_mean = torch.cat((butter_fut, smp_fut), dim=0)
            
            for i in range(num_samples):
                fut_values = self.covariance_learner.sample(1, futs_mean).squeeze(0)
                
                butter_fut_sampled = fut_values[:butter_fut.shape[0]]  # Use input shape
                smp_fut_sampled = fut_values[butter_fut.shape[0]:]      # Use input shape

                butter_samples[i] = butter_fut_sampled
                smp_samples[i] = smp_fut_sampled

                single_pred = torch.zeros(time_steps)
                for t in range(time_steps):
                    combined_features = torch.cat((butter_fut_sampled[t].unsqueeze(0), smp_fut_sampled[t].unsqueeze(0)), dim=0)

                    single_pred[t] = self.linear(combined_features).squeeze(0) # Squeeze the output

                pred_samples[i] = single_pred

            return pred_samples, butter_samples, smp_samples

    num_fut_values = len(cols_fut_true)

    # Instantiate the model and optimizer
    model = SimpleLRModel(input_dim=len(cols_x), future_dim=num_fut_values)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Training loop
    num_epochs = args.epochs

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        pred = model(x_train)

        loss = -torch.sum(model.combined_log_prob(fut_estimate_train, fut_true_train, pred, fut_target_train))
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    print("MAE of index transformation: ", np.mean(np.abs(pred[:, 0].detach().numpy() - fut_target_train[:, 0].numpy())))

    print("Testting on test set")
    with torch.no_grad():
        model.eval()

        pred_test = model(x_test)
        test_loss = -torch.sum(model.combined_log_prob(fut_estimate_test, fut_true_test, pred_test, fut_target_test))
        print(f'Test Loss: {test_loss.item():.4f}')
        print("MAE of index transformation: ", np.mean(np.abs(pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())))
        print("MSE of index transformation: ", np.mean((pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())**2))
        print("RMSE of index transformation: ", np.sqrt(np.mean((pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())**2)))


    mlflow.log_metric("MAE train", np.mean(np.abs(pred[:, 0].detach().numpy() - fut_target_train[:, 0].numpy())))
    mlflow.log_metric("MAE test", np.mean(np.abs(pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())))
    mlflow.log_metric("RMSE test", np.sqrt(np.mean((pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())**2)))
    mlflow.log_metric("MSE test", np.mean((pred_test[:, 0].detach().numpy() - fut_target_test[:, 0].numpy())**2))

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.pytorch.log_model(
        model, 
        registered_model_name=args.registered_model_name,
        artifact_path="model_lr_generative",
    )

    
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()




if __name__ == "__main__":
    main()

Overwriting ./src/main.py


As you can see in this script, once the model is trained, the model file is saved and registered to the workspace. Now you can use the registered model in inferencing endpoints.

## Configure the command

Now that you have a script that can perform the desired tasks, you'll use the general purpose **command** that can run command line actions. This command line action can be directly calling system commands or by running a script. 

Here, you'll create input variables to specify the input data, split ratio, learning rate and registered model name.  The command script will:
* Use the environment created earlier - you can use the `@latest` notation to indicate the latest version of the environment when the command is run.
* Configure some metadata like display name, experiment name etc. An *experiment* is a container for all the iterations you do on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure ML studio.
* Configure the command line action itself - `python main.py` in this case. The inputs/outputs are accessible in the command via the `${{ ... }}` notation.
* In this sample, we access the data from a file on the internet. 

In [111]:
import pandas as pd

In [112]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "pytorch_lr_model"

job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="C:/Users/TBerends/OneDrive - Geris BV/Documents/GitHub/Geris-Dairy-Solutions-BV/Trading/Risk/Flows/weekly_data.json",
        ),
        learning_rate=0.1,
        registered_model_name=registered_model_name,
        test_end="2024-01-01",
        epochs=1000,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="ml-torch-lr:1",
    experiment_name="train_model_pytorch",
    display_name="train_model_pytorch",
)

## Submit the job 

It's now time to submit the job to run in AzureML. This time you'll use `create_or_update`  on `ml_client.jobs`.

In [113]:
ml_client.create_or_update(job)

Uploading src (0.01 MBs): 100%|##########| 10089/10089 [00:00<00:00, 553574.38it/s]




Experiment,Name,Type,Status,Details Page
train_model_pytorch,bubbly_rose_prl0gj8z16,command,Starting,Link to Azure Machine Learning studio


## View job output and wait for job completion

View the job in Azure ML studio by selecting the link in the output of the previous cell. 

The output of this job will look like this in Azure ML studio. Explore the tabs for various details like metrics, outputs etc. Once completed, the job will register a model in your workspace as a result of training. 

![Screenshot that shows the job overview](media/view-job.gif "View the job in studio")

> [!IMPORTANT]
> Wait until the status of the job is complete before returning to this notebook to continue. The job will take 2 to 3 minutes to run. It could take longer (up to 10 minutes) if the compute has been scaled down to zero nodes and custom environment is still building.



## Deploy the model as an online endpoint

Now deploy your machine learning model as a web service in the Azure cloud, an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you usually need:

* The model assets (file, metadata) that you want to deploy. You've already registered these assets in your training job.
* Some code to run as a service. The code executes the model on a given input request. This entry script receives data submitted to a deployed web service and passes it to the model, then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns. With an MLFlow model, as in this tutorial, this script is automatically created for you. Samples of scoring scripts can be found [here](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online).


## Create a new online endpoint

Now that you have a registered model and an inference script, it's time to create your online endpoint. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20(UUID,%2C%20for%20practical%20purposes%2C%20unique.).

In [126]:
import uuid

# Generate a valid endpoint name
base_name = "lrpytorchgenerative"
uuid_part = str(uuid.uuid4())[:8]  # Take the first 8 characters of the UUID
online_endpoint_name = base_name + "" + uuid_part

# Step 1: Ensure the name is between 3 and 63 characters
if len(online_endpoint_name) > 63:
    online_endpoint_name = online_endpoint_name[:63]

# Step 2: Ensure it doesn't end with a hyphen
online_endpoint_name = online_endpoint_name.rstrip('-')

# Step 3: Ensure it doesn't start with a hyphen (just in case)
if online_endpoint_name.startswith('-'):
    online_endpoint_name = online_endpoint_name[1:]

# Step 4: Ensure the name only contains lowercase letters, numbers, and hyphens
valid_name = "".join(c if c.islower() or c.isdigit() or c == '-' else '' for c in online_endpoint_name)

print(f"Valid online endpoint name: {valid_name}")


Valid online endpoint name: lrpytorchgenerative63b3dedb


> [!NOTE]
> Expect the endpoint creation to take approximately 6 to 8 minutes.

In [127]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=valid_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "weekly_data",
        "model_type": "pytorch",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint lrpytorchgenerative63b3dedb provisioning state: Succeeded


Once you've created an endpoint, you can retrieve it as below:

In [128]:
endpoint = ml_client.online_endpoints.get(name=valid_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "lrpytorchgenerative63b3dedb" with provisioning state "Succeeded" is retrieved


## Deploy the model to the endpoint

Once the endpoint is created, deploy the model with the entry script. Each endpoint can have multiple deployments. Direct traffic to these deployments can be specified using rules. Here you'll create a single deployment that handles 100% of the incoming traffic. We have chosen a color name for the deployment, for example, *blue*, *green*, *red* deployments, which is arbitrary.

You can check the **Models** page on Azure ML studio, to identify the latest version of your registered model. Alternatively, the code below will retrieve the latest version number for you to use.

In [133]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=  registered_model_name) if m.name == registered_model_name]
)

Deploy the latest version of the model.  

> [!NOTE]
> Expect this deployment to take approximately 6 to 8 minutes.

In [ ]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint lrpytorchgenerative63b3dedb exists


......................

### Test with a sample query

Now that the model is deployed to the endpoint, you can run inference with it.

Create a sample request file following the design expected in the run method in the score script.

In [27]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [28]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

Writing ./deploy/sample-request.json


In [30]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'[1, 0]'

## Clean up resources

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [31]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

...

## Next Steps

Learn about creating a multi step pipeline for this script [Create production ML pipelines in a Jupyter notebook](https://github.com/Azure/azureml-examples/blob/main/tutorials/e2e-ds-experience/e2e-ml-workflow.ipynb).